In [1]:
import os, sys

def find_project_root(start_path):
    cur = os.path.abspath(start_path)
    while True:
        if os.path.isdir(os.path.join(cur, "src")):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            break
        cur = parent
    raise RuntimeError("Project root not found.")

ROOT = find_project_root(os.getcwd())
sys.path.append(ROOT)
os.chdir(ROOT)

print("ROOT:", ROOT)
print("CWD :", os.getcwd())


ROOT: c:\Users\sam\Documents\VSCode_tunnel\cv_2024_upscale
CWD : c:\Users\sam\Documents\VSCode_tunnel\cv_2024_upscale


In [2]:
from src.config.train_config import TrainConfig as cfg

keys = [
    "lr_dir", "hr_dir", "patch_size",
    "batch_size", "learning_rate", "num_epochs",
    "model_name", "checkpoint_dir", "num_workers", "save_every", "exp_name"
]

for k in keys:
    print(f"{k:15s} = {getattr(cfg, k)}")

lr_dir          = data/train_lr
hr_dir          = data/train_hr
patch_size      = None
batch_size      = 16
learning_rate   = 0.0001
num_epochs      = 50
model_name      = unet
checkpoint_dir  = models_ckpt
num_workers     = 2
save_every      = 10
exp_name        = None


In [8]:
def set_train_config(
    model="unet",
    patch=None,
    bs=16,
    lr=1e-4,
    epochs=50,
    ckpt_dir="models_ckpt",
    num_workers=2,
    save_every=10,
    exp_name=None
):
    from src.config.train_config import TrainConfig as cfg

    cfg.model_name    = model
    cfg.patch_size    = patch
    cfg.batch_size    = bs
    cfg.learning_rate = lr
    cfg.num_epochs    = epochs
    cfg.checkpoint_dir = ckpt_dir
    cfg.num_workers   = num_workers
    cfg.save_every    = save_every
    if exp_name is not None:
        cfg.exp_name = exp_name
            
    print("✅ TrainConfig updated:")
    print(f"  model       = {cfg.model_name}")
    print(f"  patch_size  = {cfg.patch_size}")
    print(f"  batch_size  = {cfg.batch_size}")
    print(f"  lr          = {cfg.learning_rate}")
    print(f"  epochs      = {cfg.num_epochs}")
    print(f"  ckpt_dir    = {cfg.checkpoint_dir}")
    print(f"  num_workers = {cfg.num_workers}")
    print(f"  save_every  = {cfg.save_every}")
    print(f"  exp_name    = {cfg.exp_name}")

In [9]:
set_train_config(
    model="unet",
    patch=None,       # None = full image, 32 = patch training
    bs=16,
    lr=1e-4,
    epochs=10,
    ckpt_dir="models_ckpt",
    num_workers=2,
    save_every=5,
)

✅ TrainConfig updated:
  model       = unet
  patch_size  = None
  batch_size  = 16
  lr          = 0.0001
  epochs      = 10
  ckpt_dir    = models_ckpt
  num_workers = 2
  save_every  = 5
  exp_name    = None


In [10]:
from src.train_for_unet import train as train_unet
from src.train_for_srcnn import train as train_srcnn

if cfg.model_name.lower() == "unet":
    train_unet()
elif cfg.model_name.lower() == "srcnn":
    train_srcnn()

🚀 Training UNetSR on cuda...
Found 492 image pairs.


Epoch 10/10: 100%|██████████| 31/31 [00:13<00:00,  2.37it/s, loss=0.086439, lr=0.000100]

🎉 UNet Training Finished! Saved to models_ckpt\unet_psfull_bs16_lr1e-4_final.pth


In [13]:
from src.eval import evaluate

model_1 = "unet_psfull_bs16_lr1e-4_final.pth"
model_2 = "unet_ps16_bs16_lr1e-4_final.pth"

print(f"Evaluating model: {model_1}")
evaluate(
    model_name="unet",
    lr_dir="data/train_lr",
    hr_dir="data/train_hr",
    checkpoint=f"models_ckpt/{model_1}",        # 舊 full-image 模型
)

print(f"\nEvaluating model: {model_2}")
evaluate(
    model_name="unet",
    lr_dir="data/train_lr",
    hr_dir="data/train_hr",
    checkpoint=f"models_ckpt/{model_2}",        # 如果你有改名，例如 unet_patch_final.pth 就填新的
)


Evaluating model: unet_psfull_bs16_lr1e-4_final.pth
Found 492 image pairs.


100%|██████████| 492/492 [00:11<00:00, 42.85it/s]


----------------------------------------
Evaluation Results for UNET:
   Average PSNR: 19.3309 dB
   Average SSIM: 0.4484
----------------------------------------

Evaluating model: unet_ps16_bs16_lr1e-4_final.pth
Found 492 image pairs.


100%|██████████| 492/492 [00:11<00:00, 42.43it/s]

----------------------------------------
Evaluation Results for UNET:
   Average PSNR: 19.3223 dB
   Average SSIM: 0.4400
----------------------------------------
